In [ ]:
import qiskit as qs
import matplotlib.pyplot as plt

In [ ]:
# Create a quantum circuit with 3 qubits
circuit = qs.QuantumCircuit(3)

In [ ]:
circuit.draw(output='mpl', style={'backgroundcolor': '#EEEEEE'})

In [ ]:
my_list = [0,1,2,8,3,2,7,1,3,4,2,9]

In [ ]:
def my_oracle(input):
    winner = 7
    if input == winner:
        return True
    else:
        return False

In [ ]:
for i, value in enumerate(my_list):
    if my_oracle(value):
        print(f"Winner found at index {i} with value {value}")
        print(f"Orcale called {i+1} times")
        break

In qunatum we can define our oracle as a operation that flips the sign of the winning value - This exists already as the control-z gate. 

In [ ]:
import qiskit as qs
from qiskit import *
import qiskit_aer as aer
import matplotlib.pyplot as plt
import numpy as np

backend = aer.StatevectorSimulator()

In [ ]:
# Define the oracle circuit
circuit = QuantumCircuit(2, name='initial')

result = backend.run(circuit).result()
sv = result.get_statevector()
sv = np.round(sv, 3)

plt.bar(range(4), (np.real(sv)), tick_label=['00', '01', '10', '11'])

In [ ]:
circuit = QuantumCircuit(2, name='initial')
circuit.h((0,1))
circuit.draw(output='mpl')

In [ ]:
result = backend.run(circuit).result()
sv = result.get_statevector()
sv = np.round(sv, 3)

plt.bar(range(4), (np.real(sv)), tick_label=['00', '01', '10', '11'])

In [ ]:
oracle = QuantumCircuit(2, name='oracle')
oracle.cz(0, 1)
oracle.draw(output='mpl')

In [ ]:
# Add the oracle to the circuit
circuit.cz(0, 1)

# Run the circuit
result = backend.run(circuit).result()
sv = result.get_statevector()
np.round(sv, 3)

plt.bar(range(4), (np.real(sv)), tick_label=['00', '01', '10', '11'])

In [ ]:
reflection = QuantumCircuit(2, name='reflection')
reflection.h([0, 1])
reflection.z([0, 1])
reflection.cz(0, 1)
reflection.h([0, 1])
reflection.to_gate(label='Reflection')
reflection.draw(output='mpl')

In [ ]:
grover_circuit = QuantumCircuit(2, 2)
grover_circuit.h([0, 1])
grover_circuit.cz(0, 1)
grover_circuit.append(reflection, [0, 1])
grover_circuit.draw(output='mpl')

In [ ]:
backend = aer.QasmSimulator()
grover_circuit = QuantumCircuit(2, 2)
grover_circuit.h([0, 1])
grover_circuit.cz(0, 1)
grover_circuit.append(reflection, [0, 1])
grover_circuit.measure([0, 1], [0, 1])
grover_circuit.draw(output='mpl')

In [ ]:
backend = aer.QasmSimulator()
grover_circuit = QuantumCircuit(2, 2)
grover_circuit.h([0, 1])
grover_circuit.cz(0, 1)
grover_circuit.h([0, 1])
grover_circuit.z([0, 1])
grover_circuit.cz(0, 1)
grover_circuit.h([0, 1])
grover_circuit.measure([0, 1], [0, 1])
grover_circuit.draw(output='mpl')

In [ ]:
result = backend.run(grover_circuit, shots=1).result()

In [ ]:
result

In [ ]:
import pandas as pd
import math

## Grover's Algorith Example

In [ ]:
# Give of list of numbers we want to find a specific winner in the list. Let us
# start by solving this problem classically.

# Here's our list
my_list = [0,1,8,3,2,7,1,3]

# Here's our winner
WINNER = 7

# We will define an oracle function that will return True if our winner is found.
def my_oracle(value):
    if value == WINNER:
        return True
    else:
        return False
    
# We can now (inefficently) loop through the list and call the oracle function
for i, value in enumerate(my_list):
    if my_oracle(value):
        print(f"Winner found at index {i} with value {value}")
        print(f"Orcale called {i+1} times")
        break

Obviously there are better algorithms for this job however classically this problem has a computational scaling of N/2. Using a qunatum algorithm we can achieve $\sqrt{N}$ scaling.

We can describe our list using a 3 quibit quantum state:

[000, 001, 010, 011, 100, 101, 110, 111]

Now our winner is the state 101.

In [ ]:
from qiskit.circuit.library import GroverOperator, MCMT, ZGate

# Our orcale is now going to be a qunatum circuit
def get_grover_oracle(winner):

    num_qbits = len(winner)

    circuit = QuantumCircuit(num_qbits, name='oracle')

    # Qiskit uses reversed bit ordering so me have to reverse the winner string
    winner = winner[::-1]

    # Find the bits that are 0
    zero_inds = [i for i, bit in enumerate(winner) if bit == '0']

    if len(zero_inds) > 0:
        # Add an x gates and a multi controlled z gate
        circuit.x(zero_inds)

    circuit.compose(MCMT(ZGate(), num_qbits-1, 1), inplace=True)
    
    if len(zero_inds) > 0:
        # Add an x gates and a multi controlled z gate
        circuit.x(zero_inds)

    return circuit

In [ ]:
# Here's our new winner
QUANTUM_WINNER = '011'

grover_oracle = get_grover_oracle(QUANTUM_WINNER)
grover_oracle.draw(output='mpl')

In [ ]:
grover_oracle = grover_oracle.to_gate(label='oracle')

In [ ]:
circuit = QuantumCircuit(len(QUANTUM_WINNER))
circuit.h(range(len(QUANTUM_WINNER)))
circuit.append(grover_oracle, range(len(QUANTUM_WINNER)))
circuit.draw(output='mpl')

In [ ]:
simulator = aer.StatevectorSimulator()
result = simulator.run(circuit.decompose()).result()
sv = np.round(result.get_statevector(), 3)

In [ ]:
state_labels = [f"{i:0{len(QUANTUM_WINNER)}b}" for i in range(2**len(QUANTUM_WINNER))]
df = pd.DataFrame(np.array([state_labels, sv]).T, columns=['State', 'Vector'])
df

We can see in the table above that the "winning" state has been "marked" by turning it negative. We can now turn this mark into a result using the grover operator.

In [ ]:
# Let's redefine our oracle function ...
grover_oracle = get_grover_oracle(WINNER)

# ... and take a look at this operator
grover_op = GroverOperator(grover_oracle)
grover_op.decompose().draw(output='mpl')

This operator is going to increase the probabilities of being in a marked state each time it is run.

In [ ]:
# There is an optimal number of iterations for Grover's algorithm. 
# This is dependent on the number of winners and the number of qubits.
optimal_num_iterations = math.floor(
    math.pi / (4 * math.asin(math.sqrt(1 / 2**grover_op.num_qubits)))
)
print(f"Optimal number of iterations: {optimal_num_iterations}")

In [ ]:
circuit = QuantumCircuit(grover_op.num_qubits)

# Create an even superposition
circuit.h(range(grover_op.num_qubits))

# Apply the Grover operator optimal_num_iterations times
for i in range(optimal_num_iterations):
    circuit.append(grover_op.decompose(), range(grover_op.num_qubits))

# Measure the qubits
circuit.measure_all()
circuit.draw(output='mpl')

In [ ]:
simulator = aer.QasmSimulator()
result = simulator.run(circuit.decompose(), shots=1000).result()
counts = result.get_counts()

plt.bar(counts.keys(), counts.values())

In [ ]:
def find_value(list, value, verbose=True):

    # Find number of quibits needed to represent the list
    num_qubits = math.ceil(math.log2(len(list)))

    # Convert the list into a list of states
    qubit_states = [f"{i:0{num_qubits}b}" for i in range(2**num_qubits)]

    grover_oracle = get_grover_oracle(value)

    grover_op = GroverOperator(grover_oracle)
    grover_op.decompose().draw(output='mpl')

In [ ]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import GroverOperator
from qiskit.quantum_info import Statevector
import numpy as np

In [ ]:
# list = [0,1,8,3,2,7,1,3,5,22,3,124,34,123,412,31,2312,331,23]
my_list = [4, 2, 7, 1, 5, 3]
target_value = 7

# Determine the number of qubits needed
n = len(my_list)
num_qubits = int(np.ceil(np.log2(n)))
num_value_qubits = int(np.ceil(np.log2(max(my_list) + 1)))

# Create a quantum circuit with the necessary qubits
qc = QuantumCircuit(num_qubits + num_value_qubits + 1)

# Apply Hadamard gates to the qubits representing indices
qc.h(range(num_qubits))

for i, value in enumerate(my_list):
    value_bin = format(value, f'0{num_value_qubits}b')
    for j, bit in enumerate(value_bin):
        if bit == '1':
            qc.x(num_qubits + j)
    qc.mcx(list(range(num_qubits)), num_qubits + num_value_qubits - 1)
    for j, bit in enumerate(value_bin):
        if bit == '1':
            qc.x(num_qubits + j)
    qc.barrier()

In [ ]:
qc.draw(output='mpl')

In [ ]:
# Compare the value in ancillary qubits with the target value
target_bin = format(target_value, f'0{num_value_qubits}b')
print(target_bin)
for i, bit in enumerate(target_bin):
    if bit == '0':
        qc.x(num_qubits + i)
qc.mcx(list(range(num_qubits, num_qubits + num_value_qubits)), num_qubits + num_value_qubits)
for i, bit in enumerate(target_bin):
    if bit == '0':
        qc.x(num_qubits + i)

qc.draw(output='mpl')